<h2 style = 'color: orange'> S8  - Machine Learning - Megaline Telecom  </h2>

In [44]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')

# ML
from sklearn import set_config
set_config(print_changed_only=False)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

<h3 style = 'color: orange'> Reading and Analysing Files:</h3>

In [45]:
df = pd.read_csv('users_behavior.csv')

In [46]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


сalls — número de chamadas

minutes — duração total da chamada em minutos

messages — número de mensagens de texto

mb_used — Tráfego de internet usado em MB

is_ultra — plano para o mês atual (Ultra - 1, Smart - 0)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [48]:
df.duplicated().sum()

0

<h3 style = 'color: orange'> Machine Learning (testing models):</h3>

In [49]:
# Dividindo os dados de origem em um conjunto de treinamento, um conjunto de validação e um conjunto de teste:

df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=54321)

df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=54321)

In [50]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [51]:
# declarando variáveis para 'features' e 'target':

train_features = df_train.drop('is_ultra', axis=1)
train_target = df_train['is_ultra']

valid_features = df_valid.drop('is_ultra', axis=1)
valid_target = df_valid['is_ultra']

test_features = df_test.drop('is_ultra', axis=1)
test_target = df_test['is_ultra']

<h3 style = 'color: orange'> Decision Tree Classifier:</h3>

In [52]:
# Definindo os valores a serem testados para cada hiperparâmetro:
criteria = ['gini', 'entropy']
min_samples_splits = [2, 3, 4, 5]
min_samples_leafs = [1, 2, 3, 4, 5]
max_depths = range(1, 16)

best_accuracy = 0
best_config = {}

for criterion in criteria:
    for min_samples_split in min_samples_splits:
        for min_samples_leaf in min_samples_leafs:
            for depth in max_depths:
                model = DecisionTreeClassifier(random_state=54321, criterion=criterion, 
                                               min_samples_split=min_samples_split, 
                                               min_samples_leaf=min_samples_leaf, 
                                               max_depth=depth)
                model.fit(train_features, train_target)
                
                valid_predictions = model.predict(valid_features)

                accuracy = accuracy_score(valid_target, valid_predictions)

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_config = {'criterion': criterion, 
                                   'min_samples_split': min_samples_split, 
                                   'min_samples_leaf': min_samples_leaf, 
                                   'max_depth': depth}

print("Melhor configuração:",end='')
print(best_config)
print("Acurácia da melhor configuração:", best_accuracy)

Melhor configuração:{'criterion': 'gini', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10}
Acurácia da melhor configuração: 0.7822706065318819


Usando o modelo 'Decision Tree' verifiquei que a configuração ideal para os hiperparâmetros (criterion, min_samples_split, min_samples_leaf e max_depth) é:

criterion= gini ;
min_samples_split' 2 ;
min_samples_leaf= 1 ;
max_depth= 10.

Essa configuração apresentou uma acurácia de aproximadamente 78%.

<h3 style = 'color: orange'> Random Forest Classifier:</h3>

In [53]:
# Utilizando o Grid Search para otimizar a busca por hiperparâmetros:

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 30, 60], 
    'criterion': ['gini', 'entropy'],  
    'max_depth': [None, 5, 15],  
    'min_samples_split': [2, 10, 15],  
    'min_samples_leaf': [1, 2, 4],  
    'max_features': ['auto', 'sqrt', 'log2'],  
    'bootstrap': [True, False]  
}

model = RandomForestClassifier(random_state=54321)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(train_features, train_target)

best_params = grid_search.best_params_

best_model = RandomForestClassifier(random_state=54321, **best_params)

best_model.fit(train_features, train_target)

accuracy = best_model.score(valid_features, valid_target)

print("Melhor configuração:", best_params)
print("Acurácia da melhor configuração:", accuracy)

Melhor configuração: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 15, 'n_estimators': 60}
Acurácia da melhor configuração: 0.7869362363919129


Usando o modelo 'Random Forest Classifier' verifiquei que a configuração ideal para os hipermarâmetros (n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap) é:

bootstrap = True ;
criterion = entropy ;
max_depth = 15 ;
max_features= sqrt ;
min_samples_leaf = 1 ;
min_samples_split = 15 ;
n_estimators = 60.

Essa configuração apresentou uma acurácia de aproximadamente 79%.


<h3 style = 'color: orange'> Logistic Regression:</h3>

In [54]:
# Testando o modelo de "Logistic Regression" :

model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(train_features, train_target )
score_train = model.score(train_features, train_target)
score_valid = model.score(valid_features, valid_target)

print("Acurácia do modelo de regressão logística no conjunto de treinamento:", score_train)
print("Acurácia do modelo de regressão logística no conjunto de validação:", score_valid)


Acurácia do modelo de regressão logística no conjunto de treinamento: 0.7131742738589212
Acurácia do modelo de regressão logística no conjunto de validação: 0.6780715396578538


O modelo de 'Logistic Regression' apresenta acurácia inferior (aprox. 68%) aos modelos testados anteriormente.

<h3 style = 'color: orange'> Machine Learning (chosen model):</h3>

Com uma acurácia tão parecida entre os modelos de Árevore de Regressão e Floresta Aleatória, poderia escolher a utilização da Árvore Aleatória pela velocidade de execução. Porém vou optar pela Floresta Aleatória pela ligeira vantagem nos resultados e tradição em acurácia. 

"Alta acurácia significa alta receita."

In [55]:
# Usando os dados de teste no modelo escolhido:

model = RandomForestClassifier(
    random_state=54321, bootstrap= True, 
    criterion= 'entropy', 
    max_depth= 15, 
    max_features= 'sqrt', 
    min_samples_leaf= 1, 
    min_samples_split= 15, 
    n_estimators= 60
    )

model.fit(train_features, train_target)
test_score = model.score(test_features, test_target)

print(f'A acurácia do modelo com o cojunto de dados de teste é: {test_score}.')


A acurácia do modelo com o cojunto de dados de teste é: 0.8180404354587869.


O modelo escolhido apresentou um resultado ainda melhor com o conjunto de teste, aproximadamente 82%. A acurácio ficou acima dos 75% propostos e assim o resultado foi alcançado.